In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_hdf('data/train_preprocessed_v4.h5', low_memory = False)
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
0,7.5,-73.978463,40.759072,-73.963577,40.774255,1,2014.0,9,6,15,1.719640,21.029044,0
1,6.5,-73.982397,40.770245,-73.972450,40.764062,1,2012.0,5,1,7,1.122217,21.940741,0
2,12.9,-74.006507,40.718808,-73.961096,40.760711,1,2009.0,5,2,20,5.210553,20.679895,0
3,20.1,-73.999242,40.734165,-73.972597,40.751228,2,2010.0,7,4,22,3.008668,21.891888,0
4,4.1,-73.977750,40.746274,-73.972760,40.755768,1,2010.0,8,2,13,0.626738,21.931951,0


In [3]:
df_test = pd.read_hdf('data/test_prepro_v4.h5', low_memory = False)
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid,key
0,-73.972484,40.742743,-73.918937,40.764496,1,2009.0,1,4,0,5.991612,16.117319,0,2009-01-01 00:01:04.0000003
1,-73.985850,40.722826,-73.986301,40.739347,1,2009.0,1,4,0,0.509264,23.237424,0,2009-01-01 00:01:26.0000001
2,-73.988917,40.740142,-73.982769,40.777291,1,2009.0,1,4,0,1.328903,23.142153,0,2009-01-01 00:04:42.0000001
3,-73.977163,40.764490,-73.914474,40.771575,1,2009.0,1,4,0,6.974103,15.690509,0,2009-01-01 00:04:54.0000001
4,-73.948849,40.778003,-73.977678,40.748692,2,2009.0,1,4,0,3.329684,19.450750,0,2009-01-01 00:04:59.0000004


In [4]:
df_test.drop(columns = ['passenger_count'], inplace = True)
df.drop(columns = ['passenger_count'], inplace = True)

In [5]:
def meet_fare(fare):
    if(fare < 2.5):
        return 2.5
    return fare
dis_vec = np.vectorize(meet_fare)
df['fare_amount'] = dis_vec(df.fare_amount)

In [5]:
from sklearn.model_selection import train_test_split
y = df['fare_amount']
X = df.drop(columns=['fare_amount'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [8]:
import lightgbm as lgb

params = {
        'learning_rate': 0.10,
        'application': 'regression',
        'max_depth': 5,
        'num_leaves': 30,
        'verbosity': -1,
        'metric': 'RMSE',
    }

In [9]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds = 10)
gbm.save_model('model_lim_features.txt')

[1]	valid_0's rmse: 42.0801
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 41.9536
[3]	valid_0's rmse: 41.8495
[4]	valid_0's rmse: 41.765
[5]	valid_0's rmse: 41.6954
[6]	valid_0's rmse: 41.6385
[7]	valid_0's rmse: 41.5913
[8]	valid_0's rmse: 41.5523
[9]	valid_0's rmse: 41.5201
[10]	valid_0's rmse: 41.4927
[11]	valid_0's rmse: 41.4709
[12]	valid_0's rmse: 41.453
[13]	valid_0's rmse: 41.4378
[14]	valid_0's rmse: 41.425
[15]	valid_0's rmse: 41.4139
[16]	valid_0's rmse: 41.4051
[17]	valid_0's rmse: 41.397
[18]	valid_0's rmse: 41.3902
[19]	valid_0's rmse: 41.3842
[20]	valid_0's rmse: 41.3788
[21]	valid_0's rmse: 41.3745
[22]	valid_0's rmse: 41.3706
[23]	valid_0's rmse: 41.3672
[24]	valid_0's rmse: 41.3632
[25]	valid_0's rmse: 41.3607
[26]	valid_0's rmse: 41.3583
[27]	valid_0's rmse: 41.3557
[28]	valid_0's rmse: 41.3539
[29]	valid_0's rmse: 41.3515
[30]	valid_0's rmse: 41.3495
[31]	valid_0's rmse: 41.3482
[32]	valid_0's rmse: 41.3459
[33]	valid_0's rmse: 41.

In [10]:
keys = df_test['key'] 
df_test.drop(columns = ['key'], inplace = True)
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,weekday,hour,distance,JFK_distance,invalid
0,-73.972484,40.742743,-73.918937,40.764496,2009.0,1,4,0,5.991612,16.117319,0
1,-73.985850,40.722826,-73.986301,40.739347,2009.0,1,4,0,0.509264,23.237424,0
2,-73.988917,40.740142,-73.982769,40.777291,2009.0,1,4,0,1.328903,23.142153,0
3,-73.977163,40.764490,-73.914474,40.771575,2009.0,1,4,0,6.974103,15.690509,0
4,-73.948849,40.778003,-73.977678,40.748692,2009.0,1,4,0,3.329684,19.450750,0


In [11]:
pred_fares = gbm.predict(df_test, num_iteration=gbm.best_iteration)
df_final = pd.DataFrame({'key':keys, 'fare_amount':pred_fares})

In [12]:
df_final.to_csv(r'predictions/lightgbm_withfeaures_v7.csv', index = False)